## Problem

**Task [[kaggle](https://www.kaggle.com/c/reface-fake-detection)]:** recognize fake videos. You need to train the binary classifier to distinguish real videos from fake ones (the provided fake data is the result of the technologies developed in Reface).

****

### What I should get?

In order to complete this stage, you should meet one of 2 conditions below:
+ either make a solution with a minimum target metric value of 0.92475
+ or be in the top 30 of all competitors.

****

### Evaluation

The evaluation metric for this competition is F1-Score, average='micro'. The F1 score, commonly used in information retrieval, measures accuracy using the statistics precision p and recall r. Precision is the ratio of true positives (tp) to all predicted positives (tp + fp). Recall is the ratio of true positives to all actual positives (tp + fn).

The F1 metric weights recall and precision equally, and a good retrieval algorithm will maximize both precision and recall simultaneously. Thus, moderately good performance on both will be favored over extremely good performance on one and poor performance on the other.

More information you can find at sklearn docs:
https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html

****

### Submission

For each filename in the test set, you must predict either this file is fake video (label 1) or this file is real video (label 0). The file should contain a header and have the following format:

```
filename,label
004582.mp4,1
003603.mp4,0
```

## Install external modules and load our data

In [ ]:
!pip install -qq av

In [ ]:
!pip install facenet-pytorch > /dev/null 2>&1
!apt install zip > /dev/null 2>&1

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!rm -rf reface-fake-detection

In [ ]:
!unzip -qq /content/drive/MyDrive/dl-creator-school/reface-fake-detection.zip -d reface-fake-detection

## Modules importing

In [ ]:
import os
import glob
import json
import cv2

import pandas as pd
import numpy as np

import torch
import torch.nn.functional as F
import torchvision

from torch import nn, optim
from torch.utils.data import sampler
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils import data
from torchvision import transforms, datasets
from torchsummary import summary

from facenet_pytorch import MTCNN

from sklearn.model_selection import train_test_split

from PIL import Image
from tqdm.notebook import tqdm
from joblib import Parallel, delayed

from typing import List, Dict, Tuple, Union, Optional
from pathlib import Path

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.rcParams['figure.dpi'] = 150

## Settings

In [ ]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(f'Running on device: {device}')

In [ ]:
PATH2PROJECT = Path('')
PATH2DRIVE = Path('/content/drive/MyDrive/dl-creator-school/')

PATH2DATA = PATH2PROJECT / 'reface-fake-detection'
PATH2TRAIN = PATH2DATA / 'train'
PATH2TEST = PATH2DATA / 'test'
# PATH2SUBMISSIONS = PATH2DRIVE / 'submissions'
# PATH2CHECKOUTS = PATH2DRIVE / 'checkouts'

PATH2RESULT = PATH2PROJECT / 'reface-fake-detection-result'
PATH2RESULT_TRAIN = PATH2RESULT / 'train'
PATH2RESULT_TEST = PATH2RESULT / 'test'

In [ ]:
# try: PATH2SUBMISSIONS.mkdir()
# except: pass
# try: PATH2CHECKOUTS.mkdir()
# except: pass
try: PATH2RESULT.mkdir()
except: pass
try: PATH2RESULT_TRAIN.mkdir()
except: pass
try: PATH2RESULT_TEST.mkdir()
except: pass

In [ ]:
SEED = 42
N_FRAMES = 8

## Training metadata

In [ ]:
meta_df = pd.read_csv(PATH2DATA / 'train.csv')
meta_df.shape

In [ ]:
meta_df.sample(n=5, random_state=SEED)

In [ ]:
meta_df.label.value_counts(normalize=True)

## Show few frames

In [ ]:
t = os.listdir(PATH2TRAIN / 'train')
temp_filename = t[np.random.randint(0, high=len(t))]
data = torchvision.io.read_video(str(PATH2TRAIN / 'train' / temp_filename), pts_unit='sec')
del t

In [ ]:
vid = data[0]
aud = data[1]

print("Video Shape: {}\nAudio Shape: {}\n".format(vid.shape, aud.shape))

In [ ]:
plt.imshow(vid[10])
plt.title(f"DeepFake Dataset: {temp_filename}; label={meta_df[meta_df.filename == temp_filename].label.values[0]}", loc='left', fontsize=8, pad=5);
plt.axis("off");

## Define Face Extractor

In [ ]:
class FaceExtractor:
    def __init__(self, detector, n_frames=None):
        """
        Parameters:
            n_frames {int} -- Total number of frames to load. These will be evenly spaced
                throughout the video. If not specified (i.e., None), all frames will be loaded.
                (default: {None})
            resize {float} -- Fraction by which to resize frames from original prior to face
                detection. A value less than 1 results in downsampling and a value greater than
                1 result in upsampling. (default: {None})
        """

        self.detector = detector
        self.n_frames = n_frames
    
    def __call__(self, filename, save_dir):
        """Load frames from an MP4 video, detect faces and save the results.

        Parameters:
            filename {str} -- Path to video.
            save_dir {str} -- The directory where results are saved.
        """

        # Create video reader and find length
        v_cap = cv2.VideoCapture(filename)
        v_len = int(v_cap.get(cv2.CAP_PROP_FRAME_COUNT))

        # Pick 'n_frames' evenly spaced frames to sample
        if self.n_frames is None:
            sample = np.arange(0, v_len)
        else:
            sample = np.linspace(0, v_len - 1, self.n_frames).astype(int)

        # Loop through frames
        for j in range(v_len):
            success = v_cap.grab()
            if j in sample:
                # Load frame
                success, frame = v_cap.retrieve()
                if not success:
                    continue
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frame = Image.fromarray(frame)
                
                save_path = os.path.join(save_dir, f'{j}.png')

                self.detector([frame], save_path=save_path)

        v_cap.release()

## Face extraction process

In [ ]:
# Load face detector
face_detector = MTCNN(margin=14, keep_all=True, factor=0.5, device=device).eval()

In [ ]:
# Define face extractor
face_extractor = FaceExtractor(detector=face_detector, n_frames=N_FRAMES)

In [ ]:
# Get the paths of all train videos
all_train_videos = glob.glob(os.path.join(PATH2TRAIN / 'train', '*.mp4'))
all_res_train_videos = os.listdir(PATH2RESULT_TRAIN)

all_train_videos = [path for path in all_train_videos if (path.split('/')[-1].split('.')[0] not in all_res_train_videos)]

In [ ]:
len(glob.glob(os.path.join(PATH2TRAIN / 'train', '*.mp4'))), len(all_train_videos)

In [ ]:
# Get the paths of all test videos
all_test_videos = glob.glob(os.path.join(PATH2TEST / 'test', '*.mp4'))
all_res_test_videos = os.listdir(PATH2RESULT_TEST)

all_test_videos = [path for path in all_test_videos if (path.split('/')[-1].split('.')[0] not in all_res_test_videos)]

In [ ]:
def compute_face_extractor(path, save_path):
    file_name = path.split('/')[-1]

    save_dir = os.path.join(save_path, file_name.split(".")[0])

    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    # Detect all faces appear in the video and save them.
    face_extractor(path, save_dir)

In [ ]:
with torch.no_grad():
    Parallel(n_jobs=-1, verbose=5)(delayed(compute_face_extractor)(path, PATH2RESULT_TRAIN) for path in tqdm(all_train_videos))

In [ ]:
with torch.no_grad():
    Parallel(n_jobs=-1, verbose=5)(delayed(compute_face_extractor)(path, PATH2RESULT_TEST) for path in tqdm(all_test_videos))

In [ ]:
meta_df.to_csv(PATH2RESULT / 'train.csv', index=False)

In [ ]:
!tar -zcvf reface-fake-detection-result.tar.gz reface-fake-detection-result

In [ ]:
!rm /content/drive/MyDrive/dl-creator-school/reface-fake-detection.zip

In [ ]:
!cp reface-fake-detection-result.tar.gz /content/drive/MyDrive/dl-creator-school/reface-fake-detection-result.tar.gz

In [ ]:
!ls /content/drive/MyDrive/dl-creator-school